# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
readfile = pd.read_csv("../output_data/.csv")
readfile.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
import gmaps
gmaps.configure(api_key=g_key)
gmaps.figure()


Figure(layout=FigureLayout(height='420px'))

In [13]:
humidity = readfile["Humidity"]
humidity

0      72
1      74
2      73
3      39
4      76
       ..
543    32
544     4
545    62
546    93
547    64
Name: Humidity, Length: 548, dtype: int64

In [14]:
maxhumidity = humidity.max()
maxhumidity

290

In [15]:
locations = readfile[["Lat", "Lng"]]
locations

,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42
...,...,...
543,33.97,36.66
544,24.01,-104.61
545,49.65,0.71
546,51.07,132.56


In [16]:
maxTemp = readfile[["Max Temp"]]
maxTemp
tempdifference = readfile.loc[(readfile["Max Temp"] > 70) & (readfile["Max Temp"] < 80)]
tempdifference

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
18,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
...,...,...,...,...,...,...,...,...,...,...
516,516,malindi,75,KE,1558378808,88,-3.22,40.12,77.00,9.17
521,521,high rock,75,BS,1558378893,83,26.63,-78.28,75.20,12.75
530,530,hirara,37,JP,1558378895,67,24.80,125.28,75.48,26.15
534,534,marzuq,84,YE,1558378896,36,14.40,46.47,79.44,6.02


In [20]:
pic = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=True, max_intensity=maxhumidity,
                                 point_radius = 10)

pic.add_layer(heat_layer)

pic

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
narrowed_df = readfile.loc[(readfile["Max Temp"] > 70) & (readfile["Max Temp"] < 80) & (readfile["Cloudiness"] < 100), :]
narrowed_df.reset_index(inplace=True)

narrowed_df.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
1,7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
2,18,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41
3,25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
4,28,28,rikitea,95,PF,1558378669,76,-23.12,-134.97,74.58,12.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotels = []

for index, row in narrowed_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params).json()
    #response = requested.json()
    try:
        hotels.append(requested['results'][0]['name'])
    except:
        hotels.append("")
narrowed_df["Hotel Name"] = hotels
narrowed_df = narrowed_df.dropna(how='any')
narrowed_df.head()

C:\Users\19803\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41,Mahebourg
2,18,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41,Vieux Grand Port
3,25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93,Kantang
4,28,28,rikitea,95,PF,1558378669,76,-23.12,-134.97,74.58,12.82,Rikitea
5,44,44,puerto ayora,26,EC,1558378762,86,-0.74,-90.35,79.00,5.99,Puerto Ayora


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_df.iterrows()]
locations = narrowed_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
pic.add_layer(markers)
pic

# Display Map

Figure(layout=FigureLayout(height='420px'))